In [1]:
from tqdm import tqdm

import pandas as pd
import numpy as np

from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import UnstructuredExcelLoader
from langchain_community.document_loaders import DataFrameLoader
from langchain.retrievers import BM25Retriever, EnsembleRetriever

In [2]:
class CFG:
    # store="프랭크버거"
    output_path = "/home/user11/beaver/data_processing/data_test"
    save_path = ""
    embedding_model="BAAI/bge-m3"
    retriever_k=4
    retriever_bert_weight=0.7
    version='11.5'
    seed=42
    
# CFG.save_path = CFG.output_path + CFG.store + "_temp.csv"

In [3]:
data_df = pd.read_excel(f'/home/user11/beaver/data/beaver_v{CFG.version}.xlsx').rename(columns = {
    '상점명' : 'store', 
    '이전 슬롯' : 'before_slot', 
    '현재 슬롯' : 'current_slot', 
    '이전 입력' : 'before_input', 
    '현재 입력' : 'current_input',
    '이전 응대' : 'before_response',
    '현재 응대' : 'current_response'
})
data_df['store'] = data_df['store'].astype('str')
data_df['store'] = data_df['store'].apply(lambda x: x.split("_")[0])

data_df = data_df.loc[:, ~data_df.columns.str.startswith('Unnamed')]
#data_df['store'] = data_df['store'].apply(lambda x: x.split("_")[0])

In [4]:
print(data_df)

      store                                        before_slot  \
0     프랭크버거     {'매장포장여부': None, '주문내역': [None], '결제수단': None}   
1     프랭크버거     {'매장포장여부': None, '주문내역': [None], '결제수단': None}   
2     프랭크버거  {'매장포장여부': None, '주문내역': [{'상품명': '베이컨 치즈 버거',...   
3     프랭크버거  {'매장포장여부': None, '주문내역': [{'상품명': '베이컨 치즈 버거 세...   
4     프랭크버거  {'매장포장여부': None, '주문내역': [{'상품명': '베이컨 치즈 버거 세...   
...     ...                                                ...   
1477  프랭크버거  {'매장포장여부': '포장', '주문내역': [{'상품명': '아메리카노', '옵션...   
1478  프랭크버거  {'매장포장여부': '포장', '주문내역': [{'상품명': '아메리카노', '옵션...   
1479  프랭크버거  {'매장포장여부': '포장', '주문내역': [{'상품명': '아메리카노', '옵션...   
1480  프랭크버거  {'매장포장여부': '포장', '주문내역': [{'상품명': '아메리카노', '옵션...   
1481  프랭크버거  {'매장포장여부': '포장', '주문내역': [{'상품명': '아메리카노', '옵션...   

                                           current_slot  \
0        {'매장포장여부': None, '주문내역': [None], '결제수단': None}   
1     {'매장포장여부': None, '주문내역': [{'상품명': '베이컨 치즈 버거',...   
2     {'매장포장여부': None, '주문내역':

In [5]:
# AST 검사
import ast
for idx, i in enumerate(data_df['current_slot']):
    i = i.replace("<|nooptions|>", "'<|nooptions|>'")
    try:
        ast.literal_eval(i)
    except:
        print(idx+2)

In [6]:
store_list = ['프랭크버거']

data_df___ = pd.DataFrame()

for store in tqdm(store_list, total=len(store_list)):
    data_df_ = data_df[data_df['store']==store]
    
    df = pd.read_excel("/home/user11/beaver/data_processing/매장정보2.xlsx")
    
    df_ = df[df['STORE_NM']==f"{store}_음성인식챗봇"]
    # df['OPTION_GROUP_NM'] = df['OPTION_GROUP_NM'].apply(lambda x: "음료 선택" if x=="음료선택" else x)
   

    df_ = df_[['PROD_NM', 'CATEGORY_NM', 'OPTION_GROUP_NM', 'ESS_YN', 'OPTION_PROD_NM']]
    df_ = df_.rename(columns = {
            'PROD_NM' : '상품명',
            'CATEGORY_NM' : '카테고리',
            'OPTION_GROUP_NM' : '옵션그룹',
            'ESS_YN' : '필수여부',
            'OPTION_PROD_NM' : '옵션명',
        })
    df_ = df_.groupby(["상품명", "카테고리", "옵션그룹", "필수여부",])["옵션명"].apply(list).reset_index()
    

    menu_str = ""
    for name, df__ in df_.groupby("카테고리"):
        menu_str += f"- {name}: "
        for i, df___ in df__.iterrows():
            menu_str += f"{df___['상품명']}, "
        menu_str += f"\n"
        
    data = {}
    for i, df__ in df_.groupby('상품명'):
        sub_data = {}
        
        for j, df___ in df__.iterrows():
            group = df___['옵션그룹']
            ess = f"[필수 주문]" if df___['필수여부'] == 'Y' else "[선택 주문]" if df___['필수여부'] == 'N' else "None"
            ''' if df___['필수여부'] == 'Y': [필수 주문]
                else:
                    if df___['필수여부'] == 'N': [선택 주문]
                    else: "None"
            ''' 
            # num = f"(최대 {df___['최대선택수']}개)" if type(df___['최대선택수'])==str else "" # Null이면 Null개
            name = f"{ess} {group}"
            sub_data[name] = df___['옵션명']
        
        if ess == "None":
            data[i] = "[옵션 없음]"
        else:    
            data[i] = str(sub_data)#.replace("'", "\"")
            
            
    df_2 = pd.DataFrame.from_dict(data, orient='index')#.to_csv(CFG.save_path)
    df_2 = df_2.reset_index()
    df_2.columns = ['상품명', '옵션']


    loader = DataFrameLoader(df_2, page_content_column="상품명")
    docs = loader.load()
    
    
    from langchain_community.embeddings import HuggingFaceBgeEmbeddings
    from langchain_community.vectorstores import FAISS

    encode_kwargs={'normalize_embeddings':True}
    model_kwargs={'device':'cpu'}

    hf = HuggingFaceBgeEmbeddings(
        model_name=CFG.embedding_model,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs,
    )
    
    db = FAISS.from_documents(docs, hf)

    db.save_local(f"{CFG.output_path}/{store}_faiss{CFG.version}")
    import pickle 
    with open(f"{CFG.output_path}/{store}_docs{CFG.version}.pkl", "wb") as f:
        pickle.dump(docs, f)
    
    # db = FAISS.load_local(f"/home/user14/beaver/log/db/{store}_faiss", hf, allow_dangerous_deserialization=True)
        
    retriever = db.as_retriever(
        search_type="similarity",
        search_kwargs={"k": CFG.retriever_k}
    )

    bm25_retriever = BM25Retriever.from_documents(docs)
    bm25_retriever.k = CFG.retriever_k

    ensemble_retriever = EnsembleRetriever(
        retrievers=[retriever, bm25_retriever],
        weights=[CFG.retriever_bert_weight, 1 - CFG.retriever_bert_weight],
    )
    
    data_df__ = data_df_[data_df_['store']==store].copy()
    print(data_df__.shape)
    
    def format_docs(docs):
        return "\n".join(f"- {doc}".replace('"', '') for doc in docs[:CFG.retriever_k])
    def invoke_format(example):
        before_slot = example['before_slot']
        
        text1 = " ".join([i.split('\'')[1] for i in before_slot.split("상품명': ")[1:]])
        text2 = "" if example['before_input']=="이전 대화 없음" else example['before_input']
        text3 = example['current_input']
        text = text1+" "+text2+" "+text3
        return text
    data_df__['retriever'] = data_df__.apply(lambda x: format_docs([f"{i.page_content}: {i.metadata['옵션']}" for i in ensemble_retriever.invoke(invoke_format(x))]), axis=1)
    data_df__['all_menus'] = data_df__.apply(lambda x: menu_str, axis=1)
    data_df___ = pd.concat([data_df___, data_df__], axis=0)

data_df___ = data_df___.reset_index(drop=True)

  0%|          | 0/1 [00:00<?, ?it/s]/home/user11/venv/beaver/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


(1478, 7)


100%|██████████| 1/1 [02:07<00:00, 127.96s/it]


In [7]:
retriever_list = [None] * len(data_df__) 

for index in range(0, 2):
    row = data_df__.iloc[index] # 해당 행
    print(row)
    formatted_input = invoke_format(row) # Doc으로 바꿔서
    '''스위트 프라이드 치킨 피자에 추가할 수 있는 옵션은 뭐가 있나요?'''
    retrieved_docs = ensemble_retriever.invoke(formatted_input) # 해당 메뉴의 유사 메뉴 k개
    '''[Document(metadata={'옵션': "{'[선택 주문] 치즈 추가 ': ['치즈 추가']}"}, page_content='스위트 프라이드 치킨 피자'), Document(metadata={'옵션': "{'[선택 주문] 치즈 추가 ': ['치즈 추가']}"}, page_content='스위트 불고기 피자'), Document(metadata={'옵션': "{'[선택 주문] 치즈 추가 ': ['치즈 추가']}"}, page_content='스위트 고구마 피자'), Document(metadata={'옵션': '[옵션 없음]'}, page_content='치즈 토핑 추가'), Document(metadata={'옵션': "{'[필수 주문] 조각 선택 ': ['2 조각', '4 조각']}"}, page_content='치킨 텐더')] '''
    formatted_docs = [f"{doc.page_content}: {doc.metadata['옵션']}" for doc in retrieved_docs] 
    '''["스위트 프라이드 치킨 피자: {'[선택 주문] 치즈 추가 ': ['치즈 추가']}", "스위트 불고기 피자: {'[선택 주문] 치즈 추가 ': ['치즈 추가']}", "스위트 고구마 피자: {'[선택 주문] 치즈 추가 ': ['치즈 추가']}", '치즈 토핑 추가: [옵션 없음]', "치킨 텐더: {'[필수 주문] 조각 선택 ': ['2 조각', '4 조각']}"] '''
    formatted_result = format_docs(formatted_docs)
    ''' - 스위트 프라이드 치킨 피자: {'[선택 주문] 치즈 추가 ': ['치즈 추가']}
        - 스위트 불고기 피자: {'[선택 주문] 치즈 추가 ': ['치즈 추가']}
        - 스위트 고구마 피자: {'[선택 주문] 치즈 추가 ': ['치즈 추가']}
        - 치즈 토핑 추가: [옵션 없음] '''  
    retriever_list[index] = formatted_result

data_df__['retriever'] = retriever_list

store                                                           프랭크버거
before_slot            {'매장포장여부': None, '주문내역': [None], '결제수단': None}
current_slot           {'매장포장여부': None, '주문내역': [None], '결제수단': None}
before_input                                                 이전 대화 없음
before_response                                              이전 대화 없음
current_input                                    매장 내 애완동물 출입이 가능합니까?
current_response                   정말 죄송하지만, 애완동물과 입장하시는건 어려울 것 같습니다.
retriever           - 프랭크 버거 세트: {'[필수 주문] 사이드 선택': ['콘 샐러드', '피넛 ...
all_menus           - 버거 단품: JG 버거, SG 불고기 버거, SG 크림 치즈 버거, 더블 치즈 ...
Name: 0, dtype: object
store                                                           프랭크버거
before_slot            {'매장포장여부': None, '주문내역': [None], '결제수단': None}
current_slot        {'매장포장여부': None, '주문내역': [{'상품명': '베이컨 치즈 버거',...
before_input                                     매장 내 애완동물 출입이 가능합니까?
before_response                    정말 죄송하지만, 애완동물과 입장하시는건 어려울 것 같습니

In [8]:
data_df___.to_csv(f'/home/user11/beaver/data_processing/dataset_v{CFG.version}.csv', index=False)

In [9]:
from datasets import Dataset 
dataset = Dataset.from_pandas(data_df___)

from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=25, shuffle=True, random_state=CFG.seed) # 클래스 비율 맞춰서 뽑아주는 코드

for i, (train_index, val_index) in enumerate(skf.split(dataset['current_input'], dataset['store'])): # 교차검증이라는데 클래스 비율 맞춰서 잘 뽑아주는거 같은데
    train_dataset = dataset.select(train_index) # 뽑기만하고 저장을 안함
    val_dataset = dataset.select(val_index) # 이러면 랜덤도 아닌데 randomseed를 정해놓고 뽑는거라
    
train_dataset.to_json(f"/home/user11/beaver/data_processing/dataset/beaver_v{CFG.version}_train.json")
val_dataset.to_json(f"/home/user11/beaver/data_processing/dataset/beaver_v{CFG.version}_val.json")

Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 223.72ba/s]


423256

In [11]:
import datasets
test_dataset = datasets.load_dataset('json', data_files=f'/home/user11/beaver/data_processing/dataset/beaver_v{CFG.version}_val.json')

Generating train split: 59 examples [00:00, 6749.69 examples/s]


In [12]:
test_dataset

DatasetDict({
    train: Dataset({
        features: ['store', 'before_slot', 'current_slot', 'before_input', 'before_response', 'current_input', 'current_response', 'retriever', 'all_menus'],
        num_rows: 59
    })
})

In [13]:
data_df__

,store,before_slot,current_slot,before_input,before_response,current_input,current_response,retriever,all_menus
0,프랭크버거,"{'매장포장여부': None, '주문내역': [None], '결제수단': None}","{'매장포장여부': None, '주문내역': [None], '결제수단': None}",이전 대화 없음,이전 대화 없음,매장 내 애완동물 출입이 가능합니까?,"정말 죄송하지만, 애완동물과 입장하시는건 어려울 것 같습니다.","- 프랭크 버거 세트: {'[필수 주문] 사이드 선택': ['콘 샐러드', '피넛 ...","- 버거 단품: JG 버거, SG 불고기 버거, SG 크림 치즈 버거, 더블 치즈 ..."
1,프랭크버거,"{'매장포장여부': None, '주문내역': [None], '결제수단': None}","{'매장포장여부': None, '주문내역': [{'상품명': '베이컨 치즈 버거',...",매장 내 애완동물 출입이 가능합니까?,"정말 죄송하지만, 애완동물과 입장하시는건 어려울 것 같습니다.",베이컨 치즈 버거 단품 하나 주세요.,라지와 레귤러 중에 버거의 사이즈를 선택해주세요.,"- 베이컨 치즈 버거 세트: {'[필수 주문] 사이드 선택': ['콘 샐러드', '...","- 버거 단품: JG 버거, SG 불고기 버거, SG 크림 치즈 버거, 더블 치즈 ..."
2,프랭크버거,"{'매장포장여부': None, '주문내역': [{'상품명': '베이컨 치즈 버거',...","{'매장포장여부': None, '주문내역': [{'상품명': '베이컨 치즈 버거 세...",베이컨 치즈 버거 단품 하나 주세요.,라지와 레귤러 중에 버거의 사이즈를 선택해주세요.,그냥 단품 말고 세트로 주세요.,"세트로 주문 도와드리겠습니다 고객님! 주문 상품의 사이즈, 사이드, 음료 옵션을 선...",None,"- 버거 단품: JG 버거, SG 불고기 버거, SG 크림 치즈 버거, 더블 치즈 ..."
3,프랭크버거,"{'매장포장여부': None, '주문내역': [{'상품명': '베이컨 치즈 버거 세...","{'매장포장여부': None, '주문내역': [{'상품명': '베이컨 치즈 버거 세...",그냥 단품 말고 세트로 주세요.,"세트로 주문 도와드리겠습니다 고객님! 주문 상품의 사이즈, 사이드, 음료 옵션을 선...",사이즈는 라지로 부탁드리고요 사이드는 프랜치 프라이 음료는 아메리카노로 주세요,말씀하신대로 추가해드렸습니다. 마지막으로 매장 포장여부와 결제수단을 말씀해주세요.,None,"- 버거 단품: JG 버거, SG 불고기 버거, SG 크림 치즈 버거, 더블 치즈 ..."
4,프랭크버거,"{'매장포장여부': None, '주문내역': [{'상품명': '베이컨 치즈 버거 세...","{'매장포장여부': '포장', '주문내역': [{'상품명': '베이컨 치즈 버거 세...",사이즈는 라지로 부탁드리고요 사이드는 프랜치 프라이 음료는 아메리카노로 주세요,말씀하신대로 추가해드렸습니다. 마지막으로 매장 포장여부와 결제수단을 말씀해주세요.,포장해주세요,결제 도와드리겠습니다! 결제 수단은 어떻게 하시겠어요?,None,"- 버거 단품: JG 버거, SG 불고기 버거, SG 크림 치즈 버거, 더블 치즈 ..."
...,...,...,...,...,...,...,...,...,...
1477,프랭크버거,"{'매장포장여부': '포장', '주문내역': [{'상품명': '아메리카노', '옵션...","{'매장포장여부': '포장', '주문내역': [{'상품명': '아메리카노', '옵션...",적당히 반반 해줘,"네, 추가로 주문하실 내용이 있나요?",지금까지 주문한 내용 보여줘봐,"네, 알겠습니다. 결제를 진행하시겠어요?<|showslot|>",None,"- 버거 단품: JG 버거, SG 불고기 버거, SG 크림 치즈 버거, 더블 치즈 ..."
1478,프랭크버거,"{'매장포장여부': '포장', '주문내역': [{'상품명': '아메리카노', '옵션...","{'매장포장여부': '포장', '주문내역': [{'상품명': '아메리카노', '옵션...",지금까지 주문한 내용 보여줘봐,"네, 알겠습니다. 결제를 진행하시겠어요?<|showslot|>",치즈볼 빼고 쉬림프 버거 세트로 변경할래,"네, 콘 치즈볼을 주문 목록에서 삭제하겠습니다. 쉬림프 버거 세트의 사이즈, 사이드...",None,"- 버거 단품: JG 버거, SG 불고기 버거, SG 크림 치즈 버거, 더블 치즈 ..."
1479,프랭크버거,"{'매장포장여부': '포장', '주문내역': [{'상품명': '아메리카노', '옵션...","{'매장포장여부': '포장', '주문내역': [{'상품명': '아메리카노', '옵션...",치즈볼 빼고 쉬림프 버거 세트로 변경할래,"네 알겠습니다. 쉬림프 버거 세트의 사이즈, 사이드, 음료를 선택해주세요.",라지로 해서 슈가볼하고 밀크쉐이크랑 줘,저희 매장에 슈가볼이라는 메뉴명은 존재하지 않습니다. 피넛 슈가볼을 추가해드릴까요?,None,"- 버거 단품: JG 버거, SG 불고기 버거, SG 크림 치즈 버거, 더블 치즈 ..."
1480,프랭크버거,"{'매장포장여부': '포장', '주문내역': [{'상품명': '아메리카노', '옵션...","{'매장포장여부': '포장', '주문내역': [{'상품명': '아메리카노', '옵션...",라지로 해서 슈가볼하고 밀크쉐이크랑 줘,저희 매장에 슈가볼이라는 메뉴명은 존재하지 않습니다. 피넛 슈가볼을 추가해드릴까요?,응 그렇게 해줘,"네, 추가로 주문하실 내용이 있으신가요?<|showslot|>",None,"- 버거 단품: JG 버거, SG 불고기 버거, SG 크림 치즈 버거, 더블 치즈 ..."
